We are going to display the ability to do RBF interpolation on a sphere using the geodesic distance as a metric.

In [1]:
# Imports
%load_ext autoreload
%autoreload 2

from MRBF import MRBF
import pygeodesic
import pygeodesic.geodesic as geodesic
import numpy as np
import vtk
from vtk_helpers import *

First just read in the sphere data file and gather its vertices and faces

In [2]:
# Read the mesh to get the points and faces of the mesh
filename = r'data/sphere.ply'
reader = vtk.vtkPLYReader()
reader.SetFileName(filename)
reader.Update()
polydata = reader.GetOutput()
points, faces = getPointsAndCellsFromPolydata(polydata)

In [3]:
print(faces)

[[  0   1   2]
 [  0   2   3]
 [  0   3   4]
 ...
 [418 421 419]
 [419 421 420]
 [420 421 391]]


Over the next couple of cells we are going to define a simple function on that sphere where it is 1 if z >= 0 and -1 otherwise 

In [4]:
zs = np.transpose(points)[2]
zmax = zs.max()
zmin = zs.min()
print(f"z_max = {zmax} and z_min = {zmin}")

z_max = 127.0 and z_min = -127.0


In [5]:
def f(points):
    return np.array([-1 if point[2] < 0 else 1 for point in points])

In [6]:
fs = f(points)

Lets visualize the function on the sphere using these next 2 cells

In [9]:
lut = vtk.vtkLookupTable()
lut.SetTableRange((-1, 1))
lut.Build()

scalar_bar = vtk.vtkScalarBarActor()
scalar_bar.SetOrientationToVertical()
scalar_bar.SetLookupTable(lut)
#print(lut)

In [10]:
polydata_actor = createPolyDataActor(polydataFromPointsAndCells(points, faces))
result = polydata_actor.GetMapper().GetInput().GetPointData().SetScalars(nps.numpy_to_vtk(fs))
dmin = fs[np.where(fs != np.inf)].min()
dmax = fs[np.where(fs != np.inf)].max()
polydata_actor.GetMapper().SetScalarRange([dmin, dmax])

In [11]:
v = Viewer()
v.addActors([polydata_actor, scalar_bar])
v.show()

Now lets do some interpolation

In [12]:
#select some indices to intepolate with, the first half that are even (very arbitary)
indicesToInterpolateWith = np.array([i for i in range(len(points))])[::10]
   

In [13]:
rbf = MRBF(points, faces, fs, indicesToInterpolateWith, kernel='multiquadric')

interpolating with 43 points
built interpolation matrix of size (43, 43), solving system. . . 
solved system of equations


In [14]:
interps = np.array([rbf.Interpolate(i) for i in range(len(points))])
print(interps)

[-1.00021884 -1.00022407 -0.99967401 -0.99998052 -0.99936892 -0.99922455
 -0.99871159 -0.99871571 -0.99945563 -0.9991885  -1.00005155 -1.00099203
 -1.00037745 -1.00036879 -0.99954343 -0.99965465 -0.99966951 -1.00025901
 -1.00071018 -1.00082437 -1.00004937 -0.9993461  -0.99946568 -0.99866693
 -0.9990743  -0.99928743 -0.99912094 -1.00008332 -0.99975607 -1.00026861
 -1.00001051 -0.99728664 -0.99792406 -0.99937391 -1.00084905 -0.99914977
 -1.00247928 -1.0040669  -1.00152892 -1.00033638 -1.00039615 -0.99706279
 -0.99636098 -0.99442929 -0.99379317 -0.99253733 -0.99384814 -0.99490552
 -0.99692653 -0.99712471 -1.00041733 -0.99974321 -1.00143233 -1.00399946
 -1.00210696 -0.99955549 -1.00069112 -0.9995257  -0.9977747  -0.99706665
 -1.00019512 -1.00197061 -1.00065493 -1.01016178 -1.0118386  -1.01059881
 -1.00894738 -1.0058463  -1.00039111 -1.00198636 -1.0005935  -1.00112738
 -0.99483434 -0.99429436 -0.98680358 -0.98183946 -0.98794453 -0.99495439
 -0.9949     -1.00244683 -1.00059203 -1.00202281 -0

In [15]:
print((interps.min(), interps.max()))

(-1.0118385950431412, 1.0128804677353496)


In [16]:
lut = vtk.vtkLookupTable()
lut.SetTableRange((interps.min(), interps.max()))
lut.Build()

scalar_bar = vtk.vtkScalarBarActor()
scalar_bar.SetOrientationToVertical()
scalar_bar.SetLookupTable(lut)
#print(lut)


In [17]:
polydata_actor = createPolyDataActor(polydataFromPointsAndCells(points, faces))
result = polydata_actor.GetMapper().GetInput().GetPointData().SetScalars(nps.numpy_to_vtk(interps))
dmin = interps[np.where(interps != np.inf)].min()
dmax = interps[np.where(interps != np.inf)].max()
polydata_actor.GetMapper().SetScalarRange([dmin, dmax])

In [18]:
v = Viewer()
v.addActors([polydata_actor, scalar_bar])
v.show()

now lets use a traditional rbf, in $\mathbb{R}^3$

In [19]:
from lazyfit.Rbf import Rbf

In [20]:
reducedPoints = np.array([points[i] for i in indicesToInterpolateWith])
reducedFs = np.array([fs[i] for i in indicesToInterpolateWith])


rbf2 = Rbf(reducedPoints, reducedFs, kernel='linear')

In [21]:
interps2 = np.array([rbf2.Interpolate(point) for point in points])

In [25]:
lut = vtk.vtkLookupTable()
lut.SetTableRange((interps2.min(), interps2.max()))
lut.Build()

scalar_bar = vtk.vtkScalarBarActor()
scalar_bar.SetOrientationToVertical()
scalar_bar.SetLookupTable(lut)
#print(lut)


In [26]:
polydata_actor = createPolyDataActor(polydataFromPointsAndCells(points, faces))
result = polydata_actor.GetMapper().GetInput().GetPointData().SetScalars(nps.numpy_to_vtk(interps2))
dmin = interps2[np.where(interps2 != np.inf)].min()
dmax = interps2[np.where(interps2 != np.inf)].max()
polydata_actor.GetMapper().SetScalarRange([dmin, dmax])

In [27]:
v = Viewer()
v.addActors([polydata_actor, scalar_bar])
v.show()